In [ ]:
# Our attempt at manually creating dummies (one hot encoding) 
# before realizing there is a get_dummies() function. 

Import libraries

import psycopg2
import numpy as np
import pandas as pd
from functools import reduce
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
np.random.seed(123)

In [ ]:
conn = psycopg2.connect(database='usaspending', user='postgres', password='gtown2019', host='127.0.0.1', port='5432')

In [ ]:
tbl_name = 'consolidated_data_2_filtered'

In [ ]:
df = pd.read_sql_query('SELECT * FROM ' + tbl_name, con=conn)
df = df[:1000]

# Remove duplicative columns

del df['modification_number'] # SQL filter 
del df['awarding_agency_code'] # SQL filter
del df['primary_place_of_performance_country_code'] # SQL filter
del df['performance_based_service_acquisition_code'] # SQL filter
# Codes for column names
del df['awarding_sub_agency_code']
del df['awarding_office_code']
del df['funding_sub_agency_code']
del df['funding_office_code']
del df['recipient_duns']
del df['award_or_idv_flag']
del df['award_type_code']
del df['type_of_contract_pricing_code']
del df['product_or_service_code']
del df['dod_claimant_program_code']
del df['naics_code']
del df['extent_competed_code']
del df['solicitation_procedures_code']
del df['type_of_set_aside']
del df['performance_based_service_acquisition']
del df['multi_year_contract_code']
del df['contracting_officers_determination_of_business_size_code']

# Remove "manually identified" unimportant columns
del df['action_date']
del df['funding_office_name']
del df['multi_year_contract']
del df['solicitation_procedures']

# Remove "future" data elements
del df['recipient_name']
del df['recipient_country_code']
del df['recipient_state_code']
del df['number_of_offers_received']
del df['extent_competed']
del df['contracting_officers_determination_of_business_size']
del df['number_of_employees']
del df['annual_revenue']

df.shape

In [ ]:
# Create conditional column to determine if set aside is YES or NO

def set_aside(c):
    if c['type_of_set_aside_code'] == 'NONE':
        return 0
    else:
        return 1

def contract_value(c):
    if c['base_and_exercised_options_value'] > 0:
        return c['base_and_exercised_options_value']
    elif c['base_and_all_options_value'] > 0:
        return c['base_and_all_options_value']
    elif c['total_dollars_obligated'] > 0:
        return c['total_dollars_obligated']
    elif c['federal_action_obligation'] > 0:
        return c['federal_action_obligation'] 
    else:
        return 0
    
df['set_aside'] = df.apply(set_aside, axis=1)
df['contract_value'] = df.apply(contract_value, axis=1)

del df['type_of_set_aside_code']
del df['base_and_exercised_options_value']
del df['base_and_all_options_value']
del df['total_dollars_obligated']
del df['federal_action_obligation']

df2 = df.dropna()
df2.shape

In [ ]:
numerical_cols = ['set_aside', 'contract_value']

In [ ]:
d_num = {}
df_num = pd.DataFrame(columns=['contract_transaction_unique_key'])
for name in numerical_cols:
    d_num[name] = df2[['contract_transaction_unique_key', name]]
    d_num[name] = pd.pivot_table(d_num[name],index=['contract_transaction_unique_key'],values=[name],
                        aggfunc=[np.mean], fill_value = 0)
    df_num = pd.merge(df_num, d_num[name], on='contract_transaction_unique_key', how='outer') 

In [ ]:
categorical_cols = ['awarding_sub_agency_name', 'awarding_office_name', 'funding_sub_agency_name', 
                    'primary_place_of_performance_state_code', 'award_type', 'type_of_contract_pricing', 
                    'product_or_service_code_description', 'dod_claimant_program_description', 'naics_description']

In [ ]:
d_cat = {}
df_cat = pd.DataFrame(columns=['contract_transaction_unique_key'])
for name in categorical_cols:
    d_cat[name] = df2[['contract_transaction_unique_key', name]]
    d_cat[name] = pd.pivot_table(d_cat[name], index=['contract_transaction_unique_key'], columns=[name], 
                        aggfunc=len, fill_value=0)
    df_cat = pd.merge(df_cat, d_cat[name], on='contract_transaction_unique_key', how='outer') 

In [ ]:
df_merged = pd.merge(df_num, df_cat, on=['contract_transaction_unique_key'], how='outer')